In [ ]:
import json
import pandas as pd
import re
import numpy as np
from sentence_transformers import SentenceTransformer

# Preprocessing the data

In [ ]:
with open('data_parasave.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df

,category,product_name,product_brand,price,promo,url,ingredients,description,scraping_date
0,Solar,NUXE SUN HUILE SOLAIRE BRONZANTE SPF50 150ML,NUXE,"117,00",0,https://www.paralabel.tn/accueil/1875-nuxe-sun...,None,Description :\nL'huile solaire bronzante SPF 5...,2026-01-29 21:33:54
1,Solar,DAYLONG AFTER SUN REPAIR 100ML,DAYLONG,"60,00",0,https://www.paralabel.tn/accueil/235-daylong-a...,None,C’est une lotion liposomale régénératrice rafr...,2026-01-29 21:33:54
2,Solar,Daylong Kids lait solaire SPF50+ 150ML,DAYLONG,"89,00",0,https://www.paralabel.tn/accueil/228-daylong-k...,None,Daylong Kids est un lait de protection solaire...,2026-01-29 21:33:54
3,Solar,LA ROCHE POSAY Anthelios Xl Gel Creme Toucher...,LA ROCHE-POSAY,"49,00",0,https://www.paralabel.tn/accueil/55-la-roche-p...,None,None,2026-01-29 21:33:54
4,Solar,FILORGA UV BRONZE 50+,FILORGA,"85,00",0,https://www.paralabel.tn/accueil/52-filorga-uv...,None,🌞 QU’EST-CE QUE C’EST ?\nFilorga UV-Bronze Flu...,2026-01-29 21:33:54
...,...,...,...,...,...,...,...,...,...
705,faceGel,algoVita Gel Nettoyant Eclaircissant Peau Sens...,ALGOVITA,"25,000DT",0,https://www.parashop.tn/visage/nettoyant-demaq...,None,DESCRIPTION\nAVIS\nalgoVita Gel Nettoyant Ecla...,2026-01-30 01:52:03
706,faceGel,algoVita Gel Nettoyant Purifiant Peau Grasse 1...,ALGOVITA,"29,000DT",0,https://www.parashop.tn/visage/nettoyant-demaq...,None,DESCRIPTION\nAVIS\nalgoVita Gel Nettoyant Puri...,2026-01-30 01:52:03
707,faceGel,AVENE CICALFATE+ GEL NETTOYANT ASSAINISSANT 200ML,AVENE,"42,450DT",1,https://www.parashop.tn/visage/nettoyant-demaq...,None,DESCRIPTION\nAVIS\nAVENE CICALFATE+ GEL NETTOY...,2026-01-30 01:52:03
708,faceGel,AVENE TOLERANCE LOTION NETTOYANTE GELIFIEE 200ML,AVENE,"38,222DT",0,https://www.parashop.tn/visage/nettoyant-demaq...,None,DESCRIPTION\nAVIS\nAVENE TOLÉRANCE LOTION NETT...,2026-01-30 01:52:03


In [ ]:
df['description'].isna().sum()

np.int64(17)

In [ ]:
df = df.dropna(subset=['description'])

In [ ]:
df['price']

,price
0,"117,00"
1,"60,00"
2,"89,00"
4,"85,00"
5,"65,00"
...,...
705,"25,000DT"
706,"29,000DT"
707,"42,450DT"
708,"38,222DT"


In [ ]:
df.duplicated().sum()

np.int64(0)

In [ ]:
df['price'] = df['price'].str.replace('DT', '').str.replace('.', '').str.replace(',', '.').astype(float)

/tmp/ipython-input-979482944.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price'] = df['price'].str.replace('DT', '').str.replace('.', '').str.replace(',', '.').astype(float)


In [ ]:
df['price']

,price
0,117.000
1,60.000
2,89.000
4,85.000
5,65.000
...,...
705,25.000
706,29.000
707,42.450
708,38.222


## Extracting the ingredients

In [ ]:
def format_ingredients_smart(text):
    """
    Smart formatting that handles multiple formats
    """
    text = text.strip()

    # Detect separator type
    newlines = text.count('\n')
    dots = text.count('.')
    commas = text.count(',')

    ingredients = []

    # Priority 1: Newline-separated (most common for your case)
    if newlines > 2:
        lines = text.split('\n')

        for line in lines:
            line = line.strip()

            # Skip empty
            if not line:
                continue

            # Skip pure descriptions in parentheses
            if re.match(r'^\([^)]+\)$', line):
                continue

            # Remove trailing parenthetical descriptions
            cleaned = re.sub(r'\s*\([^)]*\)\s*$', '', line)

            # Remove inline dashes with descriptions
            cleaned = re.sub(r'\s*-\s*[^-]+$', '', cleaned)

            cleaned = cleaned.strip()

            # Keep if it looks like an ingredient (not empty, not too long description)
            if cleaned and len(cleaned) < 100:
                ingredients.append(cleaned)

    # Priority 2: Dot-separated
    elif dots > 2:
        parts = text.split('.')
        ingredients = [p.strip() for p in parts if p.strip() and len(p.strip()) > 2]

    # Priority 3: Comma-separated
    elif commas > 2:
        parts = text.split(',')
        ingredients = [p.strip() for p in parts if p.strip() and len(p.strip()) > 2]

    # Fallback: keep as is
    else:
        ingredients = [text]

    # Join with semicolons
    result = ';'.join(ingredients) if ingredients else None
    result=result.replace('\t', ';')if result else None
    return result.replace('\n', ';') if result else None

In [ ]:
def extract_and_format_ingredients(description):
    """
    Complete solution for extracting and formatting ingredients
    """
    if not description:
        return None

    # Keywords
    keywords = ['composition', 'ingrédients', 'ingredients', 'formule', 'composants', 'inci']

    # Extract ingredients section
    for keyword in keywords:
        pattern = rf'{keyword}\s*[:：]?\s*(.*?)(?=Conseils|Mode d\'emploi|Utilisation|Précautions|Avertissement|$)'
        match = re.search(pattern, description, re.IGNORECASE | re.DOTALL)

        if match:
            ingredients_text = match.group(1).strip()

            if len(ingredients_text) > 10:
                return format_ingredients_smart(ingredients_text)

    return None

In [ ]:
df['ingredients'] = df['description'].apply(extract_and_format_ingredients)

/tmp/ipython-input-2062038096.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ingredients'] = df['description'].apply(extract_and_format_ingredients)


In [ ]:
df

,category,product_name,product_brand,price,promo,url,ingredients,description,scraping_date
0,Solar,NUXE SUN HUILE SOLAIRE BRONZANTE SPF50 150ML,NUXE,117.000,0,https://www.paralabel.tn/accueil/1875-nuxe-sun...,COCO-CAPRYLATE/CAPRATE;C12-15 ALKYL BENZOATE;D...,Description :\nL'huile solaire bronzante SPF 5...,2026-01-29 21:33:54
1,Solar,DAYLONG AFTER SUN REPAIR 100ML,DAYLONG,60.000,0,https://www.paralabel.tn/accueil/235-daylong-a...,None,C’est une lotion liposomale régénératrice rafr...,2026-01-29 21:33:54
2,Solar,Daylong Kids lait solaire SPF50+ 150ML,DAYLONG,89.000,0,https://www.paralabel.tn/accueil/228-daylong-k...,Rate…;Perfect;Good;Note;Not that bad;Very poor,Daylong Kids est un lait de protection solaire...,2026-01-29 21:33:54
4,Solar,FILORGA UV BRONZE 50+,FILORGA,85.000,0,https://www.paralabel.tn/accueil/52-filorga-uv...,None,🌞 QU’EST-CE QUE C’EST ?\nFilorga UV-Bronze Flu...,2026-01-29 21:33:54
5,Solar,Dermacare PHOTOSUN LAIT SOLAIR After sun 200 m...,DERMACARE,65.000,0,https://www.paralabel.tn/accueil/2761-dermacar...,adaptée aux peaux sensibles;Convient pour le v...,DERMACARE Photosun Pack – Lait Solaire SPF50+ ...,2026-01-29 21:33:54
...,...,...,...,...,...,...,...,...,...
705,faceGel,algoVita Gel Nettoyant Eclaircissant Peau Sens...,ALGOVITA,25.000,0,https://www.parashop.tn/visage/nettoyant-demaq...,Eau;Glycérine;Sodium cocoyl glutamate;cocamido...,DESCRIPTION\nAVIS\nalgoVita Gel Nettoyant Ecla...,2026-01-30 01:52:03
706,faceGel,algoVita Gel Nettoyant Purifiant Peau Grasse 1...,ALGOVITA,29.000,0,https://www.parashop.tn/visage/nettoyant-demaq...,Eau;hydrolat de concombre;gel d’aloe vera;coca...,DESCRIPTION\nAVIS\nalgoVita Gel Nettoyant Puri...,2026-01-30 01:52:03
707,faceGel,AVENE CICALFATE+ GEL NETTOYANT ASSAINISSANT 200ML,AVENE,42.450,1,https://www.parashop.tn/visage/nettoyant-demaq...,AVENE THERMAL SPRING WATER (AVENE AQUA);WATER ...,DESCRIPTION\nAVIS\nAVENE CICALFATE+ GEL NETTOY...,2026-01-30 01:52:03
708,faceGel,AVENE TOLERANCE LOTION NETTOYANTE GELIFIEE 200ML,AVENE,38.222,0,https://www.parashop.tn/visage/nettoyant-demaq...,AVENE THERMAL SPRING WATER (AVENE AQUA);CETEAR...,DESCRIPTION\nAVIS\nAVENE TOLÉRANCE LOTION NETT...,2026-01-30 01:52:03


In [ ]:
df.to_json('data_parasave_with_ingredients.json', orient='records', force_ascii=False)

## Filling the missing ingredients

In [ ]:
df['ingredients'].isna().sum()

np.int64(136)

In [ ]:
product_counts=df['product_name'].value_counts()

In [ ]:
product_counts=product_counts[product_counts > 1]

,count
product_name,
ALANIA UV PARFAIT Ecran Solaire Visage Teinte...,2
Heliocare 360 Pigment Solution Fluid SPF50+,2
AVENE B-PROTECT SPF50+ 30ML,2
HELIOCARE Ultra Cream SPF90 50ML,2
LA ROCHE POSAY ANTHELIOS UVMUNE 400 CREME...,2
SVR SUN SECURE BLUR SPF50+ Sans Parfum,2
SVR SUN SECURE BLUR SPF50+,2
SVR SEBIACLEAR CREME SPF50+ 40ML,2
LA ROCHE POSAY ANTHELIOS UVMUNE 400 FLUID...,2


,product_name,ingredients
58,DERMACEUTIC K CEUTIC REPARATEUR SPF 50 30ML,"Vitamine K, élastine végétale, collagène marin..."
62,AVENE B-PROTECT SPF50+ 30ML,AVENE THERMAL SPRING WATER (AVENE AQUA);C12-15...
88,ADERMA PROTECT AC Fluide Matifiant SPF50+,WATER (AQUA);C12-15 ALKYL BENZOATE;METHYLENE B...
93,BIO TACHES SOLAIRE INVISIBLE SPF50 50ML,AQUA;OCTOCRYLENE;ETHYLHEXYL METHOXYCINNAMATE;M...
94,BIO TACHES SOLAIRE TEINTE SPF 50 50ML,Aqua;octocrylene;ethylhexyl methoxycinnamate;m...
96,Heliocare 360 Mineral Tolerance Fluid SPF 50 50ML,SPF50 & AP++++;Filtres minéraux uniquement;San...
97,Heliocare 360 Water Gel SPF50+ 50ML,Ingredients: Aqua; Octocrylene; Ethylhexyl Sal...
99,SVR SUN SECURE BLUR TEINTE SPF50+,LAB21.28/A.10 - INGREDIENTS : AQUA/WATER/EAU;S...
102,ISDIN Active Unify Fusion Fluid SPF 50+ 50ML,Aqua (eau);adipate de dibutyle;cyclopentasilox...
108,Heliocare 360 Gel SPF 50+ 50ML,AQUA;ETHYLHEXYL METHOXYCINNAMATE;DIETHYLAMINO ...


In [ ]:
df = pd.DataFrame(data)
df

,category,product_name,product_brand,price,promo,url,ingredients,description,scraping_date
0,Solar,NUXE SUN HUILE SOLAIRE BRONZANTE SPF50 150ML,NUXE,117.000,0,https://www.paralabel.tn/accueil/1875-nuxe-sun...,COCO-CAPRYLATE/CAPRATE;C12-15 ALKYL BENZOATE;D...,Description :\nL'huile solaire bronzante SPF 5...,2026-01-29 21:33:54
1,Solar,DAYLONG AFTER SUN REPAIR 100ML,DAYLONG,60.000,0,https://www.paralabel.tn/accueil/235-daylong-a...,None,C’est une lotion liposomale régénératrice rafr...,2026-01-29 21:33:54
2,Solar,Daylong Kids lait solaire SPF50+ 150ML,DAYLONG,89.000,0,https://www.paralabel.tn/accueil/228-daylong-k...,Rate…;Perfect;Good;Note;Not that bad;Very poor,Daylong Kids est un lait de protection solaire...,2026-01-29 21:33:54
3,Solar,FILORGA UV BRONZE 50+,FILORGA,85.000,0,https://www.paralabel.tn/accueil/52-filorga-uv...,None,🌞 QU’EST-CE QUE C’EST ?\nFilorga UV-Bronze Flu...,2026-01-29 21:33:54
4,Solar,Dermacare PHOTOSUN LAIT SOLAIR After sun 200 m...,DERMACARE,65.000,0,https://www.paralabel.tn/accueil/2761-dermacar...,adaptée aux peaux sensibles;Convient pour le v...,DERMACARE Photosun Pack – Lait Solaire SPF50+ ...,2026-01-29 21:33:54
...,...,...,...,...,...,...,...,...,...
688,faceGel,algoVita Gel Nettoyant Eclaircissant Peau Sens...,ALGOVITA,25.000,0,https://www.parashop.tn/visage/nettoyant-demaq...,Eau;Glycérine;Sodium cocoyl glutamate;cocamido...,DESCRIPTION\nAVIS\nalgoVita Gel Nettoyant Ecla...,2026-01-30 01:52:03
689,faceGel,algoVita Gel Nettoyant Purifiant Peau Grasse 1...,ALGOVITA,29.000,0,https://www.parashop.tn/visage/nettoyant-demaq...,Eau;hydrolat de concombre;gel d’aloe vera;coca...,DESCRIPTION\nAVIS\nalgoVita Gel Nettoyant Puri...,2026-01-30 01:52:03
690,faceGel,AVENE CICALFATE+ GEL NETTOYANT ASSAINISSANT 200ML,AVENE,42.450,1,https://www.parashop.tn/visage/nettoyant-demaq...,AVENE THERMAL SPRING WATER (AVENE AQUA);WATER ...,DESCRIPTION\nAVIS\nAVENE CICALFATE+ GEL NETTOY...,2026-01-30 01:52:03
691,faceGel,AVENE TOLERANCE LOTION NETTOYANTE GELIFIEE 200ML,AVENE,38.222,0,https://www.parashop.tn/visage/nettoyant-demaq...,AVENE THERMAL SPRING WATER (AVENE AQUA);CETEAR...,DESCRIPTION\nAVIS\nAVENE TOLÉRANCE LOTION NETT...,2026-01-30 01:52:03


In [ ]:
df = df.dropna(subset=['ingredients'])

In [ ]:
df

,category,product_name,product_brand,price,promo,url,ingredients,description,scraping_date
0,Solar,NUXE SUN HUILE SOLAIRE BRONZANTE SPF50 150ML,NUXE,117.000,0,https://www.paralabel.tn/accueil/1875-nuxe-sun...,COCO-CAPRYLATE/CAPRATE;C12-15 ALKYL BENZOATE;D...,Description :\nL'huile solaire bronzante SPF 5...,2026-01-29 21:33:54
2,Solar,Daylong Kids lait solaire SPF50+ 150ML,DAYLONG,89.000,0,https://www.paralabel.tn/accueil/228-daylong-k...,Rate…;Perfect;Good;Note;Not that bad;Very poor,Daylong Kids est un lait de protection solaire...,2026-01-29 21:33:54
4,Solar,Dermacare PHOTOSUN LAIT SOLAIR After sun 200 m...,DERMACARE,65.000,0,https://www.paralabel.tn/accueil/2761-dermacar...,adaptée aux peaux sensibles;Convient pour le v...,DERMACARE Photosun Pack – Lait Solaire SPF50+ ...,2026-01-29 21:33:54
5,Solar,NUXE SUN HUILE BRONZANTE VISAGE TE CORPS SPF30...,NUXE,106.000,0,https://www.paralabel.tn/accueil/1879-nuxe-sun...,Se référer au dos des produits.,Description :\nHuile Bronzante visage et corps...,2026-01-29 21:33:54
6,Solar,SVR SUN SECURE SPRAY APRÉS SOLEIL 200ML,SVR,46.000,0,https://www.paralabel.tn/accueil/1142-svr-sun-...,Aloe vera: Apaise et hydrate la peau durableme...,SVR SUN SECURE SPRAY APRÉS SOLEIL 200ML\nSVR S...,2026-01-29 21:33:54
...,...,...,...,...,...,...,...,...,...
688,faceGel,algoVita Gel Nettoyant Eclaircissant Peau Sens...,ALGOVITA,25.000,0,https://www.parashop.tn/visage/nettoyant-demaq...,Eau;Glycérine;Sodium cocoyl glutamate;cocamido...,DESCRIPTION\nAVIS\nalgoVita Gel Nettoyant Ecla...,2026-01-30 01:52:03
689,faceGel,algoVita Gel Nettoyant Purifiant Peau Grasse 1...,ALGOVITA,29.000,0,https://www.parashop.tn/visage/nettoyant-demaq...,Eau;hydrolat de concombre;gel d’aloe vera;coca...,DESCRIPTION\nAVIS\nalgoVita Gel Nettoyant Puri...,2026-01-30 01:52:03
690,faceGel,AVENE CICALFATE+ GEL NETTOYANT ASSAINISSANT 200ML,AVENE,42.450,1,https://www.parashop.tn/visage/nettoyant-demaq...,AVENE THERMAL SPRING WATER (AVENE AQUA);WATER ...,DESCRIPTION\nAVIS\nAVENE CICALFATE+ GEL NETTOY...,2026-01-30 01:52:03
691,faceGel,AVENE TOLERANCE LOTION NETTOYANTE GELIFIEE 200ML,AVENE,38.222,0,https://www.parashop.tn/visage/nettoyant-demaq...,AVENE THERMAL SPRING WATER (AVENE AQUA);CETEAR...,DESCRIPTION\nAVIS\nAVENE TOLÉRANCE LOTION NETT...,2026-01-30 01:52:03


In [ ]:
df.to_json('data_parasave_with_ingredients.json', orient='records', force_ascii=False)

# Add Scoring

In [ ]:
with open('data_parasave_with_ingredients.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
df=pd.DataFrame(data)

In [ ]:
def parse_ingredients(ingredient_string):
    """
    Parse semicolon-separated ingredients and clean them.

    Args:
        ingredient_string: String with ingredients separated by ";"

    Returns:
        List of cleaned ingredient names
    """
    if pd.isna(ingredient_string) or ingredient_string == '':
        return []

    # Split by semicolon and clean
    ingredients = [ing.strip().lower() for ing in ingredient_string.split(';')]
    # Remove empty strings
    ingredients = [ing for ing in ingredients if ing]

    return ingredients

In [1]:
def calculate_exponential_weights(ingredients, k=0.65):
    """
    Calculate exponential decay weights for ingredients.

    Weight formula: e^(-k * position)
    Position starts at 1 for first ingredient

    Args:
        ingredients: List of ingredients in order
        k: Decay constant (default 0.65)

    Returns:
        Dictionary mapping ingredient to its weight
    """
    weights = {}
    for position, ingredient in enumerate(ingredients, start=1):
        weights[ingredient] = np.exp(-k * position)

    return weights

In [2]:

def add_ingredient_scores(df, ingredient_column='ingredients', k=0.65):
  scores_list = []

  for idx, row in df.iterrows():
    ingredients = parse_ingredients(row[ingredient_column])
    if ingredients:
        weights = calculate_exponential_weights(ingredients, k=k)
        scores_list.append(weights)
    else:
        scores_list.append({})
  #print(scores_list)

  df['ingredient_scores'] = scores_list
  return df

In [ ]:
df_result = add_ingredient_scores(df, ingredient_column='ingredients', k=0.65)
df_result

,category,product_name,product_brand,price,promo,url,ingredients,description,scraping_date,ingredient_scores
0,Solar,NUXE SUN HUILE SOLAIRE BRONZANTE SPF50 150ML,NUXE,117.000,0,https://www.paralabel.tn/accueil/1875-nuxe-sun...,COCO-CAPRYLATE/CAPRATE;C12-15 ALKYL BENZOATE;D...,Description :\nL'huile solaire bronzante SPF 5...,2026-01-29 21:33:54,"{'coco-caprylate/caprate': 0.522045776761016, ..."
1,Solar,Daylong Kids lait solaire SPF50+ 150ML,DAYLONG,89.000,0,https://www.paralabel.tn/accueil/228-daylong-k...,Rate…;Perfect;Good;Note;Not that bad;Very poor,Daylong Kids est un lait de protection solaire...,2026-01-29 21:33:54,"{'rate…': 0.522045776761016, 'perfect': 0.2725..."
2,Solar,Dermacare PHOTOSUN LAIT SOLAIR After sun 200 m...,DERMACARE,65.000,0,https://www.paralabel.tn/accueil/2761-dermacar...,adaptée aux peaux sensibles;Convient pour le v...,DERMACARE Photosun Pack – Lait Solaire SPF50+ ...,2026-01-29 21:33:54,{'adaptée aux peaux sensibles': 0.522045776761...
3,Solar,NUXE SUN HUILE BRONZANTE VISAGE TE CORPS SPF30...,NUXE,106.000,0,https://www.paralabel.tn/accueil/1879-nuxe-sun...,Se référer au dos des produits.,Description :\nHuile Bronzante visage et corps...,2026-01-29 21:33:54,{'se référer au dos des produits.': 0.52204577...
4,Solar,SVR SUN SECURE SPRAY APRÉS SOLEIL 200ML,SVR,46.000,0,https://www.paralabel.tn/accueil/1142-svr-sun-...,Aloe vera: Apaise et hydrate la peau durableme...,SVR SUN SECURE SPRAY APRÉS SOLEIL 200ML\nSVR S...,2026-01-29 21:33:54,{'aloe vera: apaise et hydrate la peau durable...
...,...,...,...,...,...,...,...,...,...,...
552,faceGel,algoVita Gel Nettoyant Eclaircissant Peau Sens...,ALGOVITA,25.000,0,https://www.parashop.tn/visage/nettoyant-demaq...,Eau;Glycérine;Sodium cocoyl glutamate;cocamido...,DESCRIPTION\nAVIS\nalgoVita Gel Nettoyant Ecla...,2026-01-30 01:52:03,"{'eau': 0.522045776761016, 'glycérine': 0.2725..."
553,faceGel,algoVita Gel Nettoyant Purifiant Peau Grasse 1...,ALGOVITA,29.000,0,https://www.parashop.tn/visage/nettoyant-demaq...,Eau;hydrolat de concombre;gel d’aloe vera;coca...,DESCRIPTION\nAVIS\nalgoVita Gel Nettoyant Puri...,2026-01-30 01:52:03,"{'eau': 0.522045776761016, 'hydrolat de concom..."
554,faceGel,AVENE CICALFATE+ GEL NETTOYANT ASSAINISSANT 200ML,AVENE,42.450,1,https://www.parashop.tn/visage/nettoyant-demaq...,AVENE THERMAL SPRING WATER (AVENE AQUA);WATER ...,DESCRIPTION\nAVIS\nAVENE CICALFATE+ GEL NETTOY...,2026-01-30 01:52:03,{'avene thermal spring water (avene aqua)': 0....
555,faceGel,AVENE TOLERANCE LOTION NETTOYANTE GELIFIEE 200ML,AVENE,38.222,0,https://www.parashop.tn/visage/nettoyant-demaq...,AVENE THERMAL SPRING WATER (AVENE AQUA);CETEAR...,DESCRIPTION\nAVIS\nAVENE TOLÉRANCE LOTION NETT...,2026-01-30 01:52:03,{'avene thermal spring water (avene aqua)': 0....


In [ ]:
df.columns

Index(['category', 'product_name', 'product_brand', 'price', 'promo', 'url',
       'ingredients', 'description', 'scraping_date', 'ingredient_scores',
       'dense_vector'],
      dtype='object')

In [ ]:
df.to_json('data_parasave_with_ingredient_scores.json', orient='records', force_ascii=False)

# Embeddings

In [ ]:
with open('data_parasave_with_ingredient_scores.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
df=pd.DataFrame(data)

## Dense vector (raw ingredients)

In [ ]:
from sentence_transformers import SentenceTransformer

#model = SentenceTransformer('all-MiniLM-L6-v2') #juste anglais
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2') #+50 langue

# Generate dense embeddings
dense_vectors = model.encode(
    df['ingredients'].tolist(),
    show_progress_bar=True,
    convert_to_numpy=True,
    batch_size=32
)

# Add to dataframe
df['dense_vector'] = dense_vectors.tolist()

print(f"Created {len(dense_vectors)} dense vectors")
print(f"Dimension: {dense_vectors.shape[1]}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/18 [00:00<?, ?it/s]

✅ Created 557 dense vectors
   Dimension: 384


## Sparse vector (ingredent scoring)

In [3]:
def create_unified_vocabulary(df, score_column='ingredient_scores'):
    """
    Create a sorted list of all unique ingredients across all products.

    Args:
        df: DataFrame with ingredient_scores column
        score_column: Column containing ingredient score dictionaries

    Returns:
        Sorted list of unique ingredients
    """
    all_ingredients = set()

    for scores_dict in df[score_column]:
        if scores_dict:
            all_ingredients.update(scores_dict.keys())

    vocabulary = sorted(all_ingredients)
    print(f"\n🔍 Created vocabulary with {len(vocabulary)} unique ingredients")

    return vocabulary

In [ ]:
def convert_scores_to_sparse_vectors(df, score_column='ingredient_scores', vocabulary=None):
    """
    Convert ingredient score dictionaries to sparse vectors.

    Args:
        df: DataFrame with ingredient_scores column
        score_column: Column containing ingredient score dictionaries
        vocabulary: List of all unique ingredients (if None, will create it)

    Returns:
        Tuple of (sparse_vectors_list, vocabulary)
    """
    print("\n" + "=" * 80)
    print("CONVERTING TO SPARSE VECTORS")
    print("=" * 80)

    # Create vocabulary if not provided
    if vocabulary is None:
        vocabulary = create_unified_vocabulary(df, score_column)

    # Convert each product to sparse vector
    sparse_vectors = []

    for idx, scores_dict in enumerate(df[score_column]):
        # Create vector: position i = score for vocabulary[i]
        vector = [scores_dict.get(ing, 0.0) for ing in vocabulary]
        sparse_vectors.append(vector)

        if idx == 0:
            # Show example for first product
            non_zero = sum(1 for v in vector if v > 0)
            print(f"\nExample - First product:")
            print(f"  Vector dimension: {len(vector)}")
            print(f"  Non-zero elements: {non_zero} ({non_zero/len(vector)*100:.1f}%)")
            print(f"  First 5 non-zero values: {[v for v in vector if v > 0][:5]}")

    print(f"\n✓ Created {len(sparse_vectors)} sparse vectors")
    print(f"  Dimension: {len(vocabulary)}")
    print(f"  Average sparsity: {np.mean([sum(1 for v in vec if v > 0) for vec in sparse_vectors]):.1f} non-zero elements")
    print("=" * 80)

    return sparse_vectors, vocabulary

In [ ]:
sparse_vectors, vocabulary = convert_scores_to_sparse_vectors(df)


CONVERTING TO SPARSE VECTORS

🔍 Created vocabulary with 2809 unique ingredients

Example - First product:
  Vector dimension: 2809
  Non-zero elements: 19 (0.7%)
  First 5 non-zero values: [0.0202419114, 0.272531793, 8.2938e-06, 0.5220457768, 0.0028798992]

✓ Created 557 sparse vectors
  Dimension: 2809
  Average sparsity: 14.4 non-zero elements


In [ ]:
df['sparse_vector'] = sparse_vectors

In [ ]:
df.attrs['vocabulary'] = vocabulary

In [ ]:
len(df.attrs['vocabulary'])

2809

In [ ]:
df.to_pickle('parasave_embedded.pkl')
with open('parasave_vocabulary.json', 'w', encoding='utf-8') as f:
    json.dump(df.attrs['vocabulary'], f, ensure_ascii=False, indent=2)